<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/VectorBased_Persistence_LLAMA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import HuggingFacePipeline


In [6]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

<ipython-input-6-a34a09b47e7d>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
!pip install langchain_chroma -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00


In [8]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='history_db',
                   embedding_function=embeddings)

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import VectorStoreRetrieverMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda



In [11]:
!pip install -Uq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.7 MB/s eta 0:00:00


In [12]:
import os
os.environ["GROQ_API_KEY"]="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"

In [13]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")

In [26]:
SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# load 2 most similar conversation messages from vector db history for each new message \ prompt
# this uses cosine embedding similarity to load the top 2 similar messgages to the input prompt \ query
retriever = chroma_db.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 15})
memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)

# creating our conversation chain now
def get_memory_messages(query):
    return [memory.load_memory_variables(query)['history']]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    llm # generates response using the prompt from previous step
)

In [27]:
query = {'query': 'Tell me about AI'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

**What is AI?**
Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception.

**Types of AI:**

1. **Narrow or Weak AI**: Designed to perform a specific task, like image recognition or language translation.
2. **General or Strong AI**: A hypothetical AI that possesses human-like intelligence and can perform any intellectual task.
3. **Superintelligence**: A hypothetical AI that significantly surpasses human intelligence in various domains.

**AI Applications:**

1. **Virtual Assistants**: Siri, Alexa, Google Assistant.
2. **Image Recognition**: Facial recognition, object detection, self-driving cars.
3. **Natural Language Processing**: Language translation, text summarization, chatbots.
4. **Predictive Maintenance**: Predicting equipment failures in industries like manufacturing and healthcare.
5. **Healthcare**: Diagnosis, treatment planni

In [16]:
query = {'query': 'What about deep learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

**What is Deep Learning?**
Deep learning is a subset of machine learning that uses neural networks with multiple layers to learn complex patterns in data. These neural networks are inspired by the structure and function of the human brain, with each layer processing and transforming the input data in a hierarchical manner.

**Types of Deep Learning:**

1. **Feedforward Neural Networks**: Data flows only in one direction, from input to output.
2. **Convolutional Neural Networks (CNNs)**: Designed for image and video processing, using convolutional and pooling layers.
3. **Recurrent Neural Networks (RNNs)**: Suitable for sequential data, such as speech, text, and time series data.
4. **Generative Adversarial Networks (GANs)**: Trained on unsupervised data to generate new, synthetic data.

**Deep Learning Applications:**

1. **Image Recognition**: Facial recognition, object detection, image classification.
2. **Natural Language Processing**: Language translation, text summarization, senti

In [17]:
query = {'query': 'Tell me about the fastest animal in the world in 2 lines'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The fastest animal in the world is the **peregrine falcon**, which can reach speeds of up to **389 km/h (242 mph)** during its characteristic hunting dive, known as a stoop. This bird's incredible acceleration allows it to catch its prey, usually small birds, in mid-air with remarkable precision and speed.


In [18]:
query = {'query': 'What about the cheetah?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The **cheetah** is indeed another contender for the title of the fastest land animal, reaching speeds of up to **120 km/h (75 mph)**. However, when it comes to the fastest animal overall, the peregrine falcon takes the top spot with its incredible diving speeds. Cheetahs are still incredibly agile and quick, though, with the ability to accelerate from 0 to 100 km/h (0-62 mph) in just 3 seconds.


Now for a new query around machine learning even if the most recent conversation messages have been about animals, it uses the vector databases to load the last 2 historical conversations which are closest to the current question in terms of semantic similarity.


In [19]:
print(memory.load_memory_variables({'query': 'What about machine learning?'})['history'])

query: Tell me about AI
output: **What is AI?**
Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception.

**Types of AI:**

1. **Narrow or Weak AI**: Designed to perform a specific task, like image recognition or language translation.
2. **General or Strong AI**: A hypothetical AI that possesses human-like intelligence and can perform any intellectual task.
3. **Superintelligence**: A hypothetical AI that significantly surpasses human intelligence in various domains.

**AI Applications:**

1. **Virtual Assistants**: Siri, Alexa, Google Assistant.
2. **Image Recognition**: Facial recognition, object detection, self-driving cars.
3. **Natural Language Processing**: Language translation, text summarization, chatbots.
4. **Predictive Maintenance**: Predicting equipment failures in industries like manufacturing and healthcare.
5. **Healthcar

In [20]:
query = {'query': 'What about machine learning?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

**What is Machine Learning?**
Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data, make predictions, or take actions without being explicitly programmed. Machine learning models can improve their performance over time by adjusting their parameters based on the data they receive.

**Types of Machine Learning:**

1. **Supervised Learning**: The algorithm learns from labeled data to make predictions or classifications.
2. **Unsupervised Learning**: The algorithm learns from unlabeled data to identify patterns or groupings.
3. **Reinforcement Learning**: The algorithm learns from interactions with an environment to maximize rewards or minimize penalties.
4. **Semi-Supervised Learning**: The algorithm combines labeled and unlabeled data to improve performance.

**Machine Learning Algorithms:**

1. **Linear Regression**: Models the relationship between a dependent variable and one or more independent variables.
2. **Decision Trees

In [21]:
query = {'query': 'what topics have we discussed, show briefly as bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Here are the topics we've discussed:

* **What is AI?**
* **Types of AI**
* **AI Applications**
* **AI Techniques**
* **Benefits and Challenges**
* **Current State of AI**
* **What is Machine Learning?**
* **Types of Machine Learning**
* **Machine Learning Algorithms**
* **Machine Learning Applications**
* **Machine Learning Techniques**
* **Benefits and Challenges**
* **Current State of Machine Learning**


In [22]:
query = {'query': 'Explain AI in 3 bullets to a child'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Here are 3 simple bullets to explain AI to a child:

* **AI is like a super helper**: It's a computer program that can do things on its own, like play games, show you pictures, or even talk to you like a friend!
* **AI learns from us**: Just like how you learn new things in school, AI learns from the things we show it, like pictures or words. It gets better and better at doing things the more it learns!
* **AI can do things we can't**: AI is really good at doing things that are hard for humans, like recognizing pictures or understanding what we say. It's like having a magic assistant that can help us with lots of things!


In [23]:
query = {'query': 'Now do the same for Generative AI'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

**What is Generative AI?**
Generative AI refers to a type of artificial intelligence that can create new and original content, such as images, music, videos, and text, based on patterns and structures learned from existing data.

**How does Generative AI work?**

1. **Training**: Generative AI models are trained on large datasets of existing content, such as images or text.
2. **Pattern recognition**: The models learn to recognize patterns and structures within the data.
3. **Content generation**: The models use the learned patterns to generate new and original content.

**Types of Generative AI:**

1. **Generative Adversarial Networks (GANs)**: A type of neural network that consists of two components: a generator and a discriminator.
2. **Variational Autoencoders (VAEs)**: A type of neural network that learns to compress and reconstruct data.
3. **Transformers**: A type of neural network that uses self-attention mechanisms to process sequential data.

**Applications of Generative AI:*

In [29]:
query = {'query': 'Give me in 3 bullets only'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Here are 3 bullets about Generative AI:

* **What is Generative AI?**: Generative AI refers to a type of artificial intelligence that can create new and original content, such as images, music, videos, and text, based on patterns and structures learned from existing data.
* **Applications of Generative AI**: Generative AI has various applications, including art and design, content creation, virtual try-on, data augmentation, and personalization, such as generating personalized content, like product recommendations.
* **Benefits and Challenges**: Generative AI offers benefits like increased creativity, efficiency, and personalization, but also raises concerns around intellectual property, authorship, and ensuring diversity, accuracy, and fairness in generated content.


In [28]:
query = {'query': 'What all topics I discussed today?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

You have discussed the following topics today:

* Showing a list of previous topics we had discussed
* Explaining AI in 3 simple bullets to a child
* Explaining AI in detail
* Explaining Deep Learning in detail
* Explaining Machine Learning in detail
* Explaining Generative AI in detail
* Asking about the fastest animal in the world
* Discussing the speed of the peregrine falcon and the cheetah


In [30]:
query = {'query': 'Tell me more about CNN in Deep Learing context ?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

**Convolutional Neural Networks (CNNs) in Deep Learning**

Convolutional Neural Networks (CNNs) are a type of deep learning model that is particularly well-suited for image and video processing tasks. They are designed to mimic the way the human brain processes visual information, by using convolutional and pooling layers to extract features from images.

**Key Components of CNNs:**

1. **Convolutional Layers**: These layers apply filters to the input image to extract features at different scales and orientations. The filters are applied in a sliding window manner, scanning the entire image to capture local patterns.
2. **Pooling Layers**: These layers reduce the spatial dimensions of the feature maps, while maintaining the most important information. This helps to reduce the number of parameters and computations required.
3. **Activation Functions**: These functions introduce non-linearity to the model, allowing it to learn more complex representations of the data. Common activation f

In [31]:
query = {'query': 'Tell me more about cricket world cup ?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

**What is the Cricket World Cup?**

The ICC Cricket World Cup is an international cricket tournament held every four years, featuring the top national teams from around the world. The tournament is organized by the International Cricket Council (ICC) and is considered the premier international cricket competition.

**History of the Cricket World Cup**

The first Cricket World Cup was held in 1975 in England, and it was won by the West Indies team. Since then, the tournament has been held every four years, with the exception of 1996, when it was held in India, Pakistan, and Sri Lanka.

**Format of the Cricket World Cup**

The Cricket World Cup typically features 10-12 teams competing in a round-robin group stage, followed by a knockout stage. The top four teams from each group qualify for the knockout stage, which consists of quarterfinals, semifinals, and the final.

**Notable Moments in Cricket World Cup History**

* 1983: India wins its first World Cup title, led by Kapil Dev.
* 1996

In [33]:
query = {'query': 'What are the questions I asked for animals so far ?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

You have asked the following questions related to animals:

1. The fastest animal in the world.
2. The speed of the peregrine falcon and the cheetah.
3. What about the cheetah?


In [34]:
query = {'query': 'What are the questions I asked on Artificial Intelligence so far ?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Here are the questions you have asked about Artificial Intelligence (AI) so far:

1. What topics have we discussed, show briefly as bullet points
2. Tell me about AI
3. Explain AI in 3 bullets to a child
4. What all topics I discussed today?
5. What are the questions I asked for animals so far?
6. What about machine learning?
7. Now do the same for Generative AI
8. What all topics I discussed today?
9. What are the challenges in CNN we discussed earlier ?
10. Tell me more about CNN in Deep Learing context ?
11. Give me in 3 bullets only
12. Tell me about the fastest animal in the world in 2 lines
